<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net_dualis_extend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
#emoji_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5e7d7d29f58be438f33fa0c0bc2fd251dea0b453/emoji.py"
#!rm emoji.py
#!wget $emoji_url
#import emoji

In [4]:
! pip install emoji
import emoji

     |████████████████████████████████| 168 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=408e05db5fe76e9ba755dbbbbfebb8f9defa9548f0206a0bf5a223f286337ac9
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d74361873258c5e8649b7f11b2ccce49
Successfully built emoji


In [5]:

emoji.demojize("hello 👍")

'hello :thumbs_up:'

##Szószedet

In [6]:
!rm SampleSubmission*
!rm TEST*
!rm TRAIN*
!rm *zip*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip
df=pd.read_csv("TEST.csv")

rm: cannot remove 'SampleSubmission*': No such file or directory
rm: cannot remove 'TEST*': No such file or directory
rm: cannot remove 'TRAIN*': No such file or directory
rm: cannot remove '*zip*': No such file or directory
--2021-10-10 20:42:26--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-10 20:42:27--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubus

In [7]:
df.head()
alltext1=df.text
print (len(alltext1))

2640


In [8]:
df2=pd.read_csv("TRAIN.csv")
alltext2=df2.text
print (len(alltext2))

12000


In [9]:
alltext1=list(alltext1)
alltext2=list(alltext2)
alltext=alltext1+alltext2
print(f" {len(alltext)}")


 14640


In [10]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
print(spell.correction("@frustrating"))

     |████████████████████████████████| 2.7 MB 4.2 MB/s 
frustrating


In [11]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def is_number(inp:str):
    a=re.search("[\d]",inp)
    if a==None: 
        return(False)
    return(True)

def strip_all_entities(text):
    import emoji
    import re,string
    entity_prefixes = ['“','@','#',"!","?",":","_"]
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in ['“','@','#',"!","?"]:
                words.append(word)

    return ' '.join(words)

def de_ex(txt):
    txt=txt.lower()
    exwords={"it's":"it is",
    "aren't":"are not",
    "i've":"i have",
    "can't":"can not",
    "didn't":"did not",
    "i'm":"i am",
    "that's":"that is",
    "let's":" let is",
    "she's":"she is",
    "we're":"we are",
    "they've":"they have",
    "fleet's":"fleet is ",
    "there's":"there is",
    "i'll": "i will"
    }
    txt_list=txt.split()
    for w in txt_list:
        if w in exwords:
            txt=txt.replace(w,exwords[w])
    return (txt)



def query_sentence(index):
    print(f"orig: {alltext[index]}")
    spell=SpellChecker()
    inp=alltext[index]
    inp=remove_URL(inp)
    inp=emoji.demojize(inp)
    inp=de_ex(inp)
    print("demojize: ",inp)
    inp=strip_all_entities(inp)
    print("STRIP: ",inp)
    txt=inp.replace(","," ").replace("."," ").replace("!"," _exclamation_ ").replace("#"," ").replace("???","?").replace("??","?").replace("?"," _questionmark_ ").replace("  "," ").replace("  "," ").replace('"',"").replace("“","")
    print("after replace=",txt)
    #print(txt)
    normal=""
    for i in txt.split(" "):
        w1=i.lower().strip()
        if "_" in w1 or ":" in w1:
            w2=w1
        else:
            if w1 !="":
                w2=spell.correction(w1).strip()
            else:
                w2=""
            if w2=="i":
                pass
                #print(f"chek: {w1}, {w2}")
            if is_number(w2):
                w2=""    
        normal += w2+" "
    normal=normal.replace("::",": :")
    print(f"  conv: {normal}")
    return(normal)

In [38]:
df2.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570018445695148033,negative,1.0,Customer Service Issue,0.656,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN


# az ötlet: A teszt adatbázisból ki lehet szűrni a negatív eseteket.
### csak a szöveg kell az eredeti adatbázisból
### a test adatokat negatívra vesszük.

Persze ha továbbgondoljuk, akkor lehetne egy (positive, neutral) selectort tanítani. Hiszen a negatív kommentekről mindent tudunk.
Ezeket könnyü is kiválogatni a rendszerből, kérdés hogy rakjuk majd össze. 
Ezt majd megoldjuk.
A legfontosabb, hogy tanulna egy ilyen "duális kimenetre a rendszerünk"


In [110]:
def listing(dfx,printing=False):
    t_selector=[]
    count_neg=0
    count_pos=0
    for i in range(len(dfx)):
    
        reason=dfx.loc[i].negativereason
        airline_sentiment="NONE"
        if "airline_sentiment" in dfx.columns:
            airline_sentiment=dfx.loc[i].airline_sentiment
            
        txt=dfx.loc[i].text
        cond=[reason=="Flight Booking Problems", 
                    reason=="Bad Flight", 
                    reason=="Late Flight", 
                    reason=="Lost Luggage", 
                    reason=="Customer Service Issue",
                    reason=="Cancelled Flight",
                    reason=="Flight Attendant Complaints",
                    reason=="longlines",
                    reason=="Damaged Luggage",
                    reason=="Can't Tell"
                    ]
        if any(cond):
            count_neg+=1
            t_selector.append(True) # kell ez az adat nekünk
            if printing:
                print(f"negative:{count_neg} --  {reason}  -- {airline_sentiment} -- {txt}" )
        else:
            count_pos+=1
            if printing:
                print(f"??:{count_pos} --  {reason}  -- {airline_sentiment} -- {txt}" )
            t_selector.append(False) # ez nem kell nem negatív
            
    print(f"Summa: negative: {count_neg}   --  positive: {count_pos}")
    return(t_selector)

In [111]:
print (len(df2))

12000


In [112]:
selector=listing(df)

KeyboardInterrupt: ignored

In [ ]:
print (len(selector))

In [ ]:
df_negativ_test=df[selector]

##Mentés

In [ ]:
t=[]
for i in range(len(alltext)):
    print(f"{i}", end=" ")
    t.append(query_sentence(i))


In [ ]:
s=set()
for sent in t:
    wlist=sent.strip().split()
    for w in wlist:
        s.add(w)
print (len(s))
ordered=list(s)
ordered.sort()

In [ ]:
print(ordered)

In [ ]:
sentence=pd.DataFrame(t)
sentence.columns=["Sentence"]
sentence.to_csv("Sentences_emoji.csv")

In [ ]:
wordlist=pd.DataFrame(ordered)
wordlist.columns=["Words"]
wordlist.to_csv("words_emoji.csv")

###---------------------------------


###Adatbetöltés SZAVAK BETÖLTÉSE

In [ ]:
import pandas as pd

In [ ]:
#words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv"


In [ ]:
!rm words* 
!wget $words_url

In [ ]:
words=pd.read_csv("words_emoji_3.csv",index_col=0)
words.head()
words_list=list(words["Words"])

In [ ]:
word_dict={v:i+1 for i,v in enumerate(words_list)}

### adatbetöltés mondatok betöltése

In [ ]:
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/Sentences.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/b9567d54968bba4d03ce8c3cac5ecdc4fe1c6031/Sentences_emoji.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv"


In [ ]:
!rm Sentences*
!wget $sentences_url
sentences=pd.read_csv("Sentences_emoji_3.csv",index_col=0)
sentences.head()
sentences_list=list(sentences["Sentence"])

##Tanulás

In [ ]:
!rm *zip*
!rm TEST*
!rm TRAIN*
!rm SampleSubmission*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip


In [ ]:
df=pd.read_csv("TRAIN.csv")

In [ ]:
df.head(4)

In [ ]:
for i in range(len(df)):
    if "😊" in df.loc[i].text:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

In [ ]:
for i in range(len(df)):
    if df.loc[i].retweet_count>0:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

In [ ]:
sentences_list[10420]

In [ ]:
df.head(1)

In [ ]:
df["clean_text"]=sentences_list[2640:]

In [114]:
df.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
0,570018445695148033,negative,1.0,Customer Service Issue,0.656,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN,your customer service is terrible _exclamation...


In [117]:
df_pos_neutral=df[(df["airline_sentiment"]=="positive") | (df["airline_sentiment"]=="neutral")]

In [118]:
df_pos_neutral.head(4)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
1,568830255282040832,neutral,1.0000,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana,i will do you now
3,569936640669581313,positive,1.0000,NaN,NaN,United,NaN,koploperfan1992,NaN,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,NaN,look at this beauty :winking_face: do united ...
4,569542242983124992,positive,1.0000,NaN,NaN,Delta,NaN,sandyy889,NaN,0,@JetBlue Landing! As usual great flight wiyh a...,NaN,2015-02-22 09:00:26 -0800,NaN,America/Detroit,landing _exclamation_ as usual great flight wi...
5,567738192426590209,neutral,0.6631,NaN,0.0,United,NaN,rajuchinthala,NaN,0,Thx! Stand by! “@united: @rajuchinthala I know...,NaN,2015-02-17 09:31:47 -0800,Indianapolis,Eastern Time (US & Canada),the _exclamation_ stand by _exclamation_ i kno...


In [ ]:
def print_df(df,start=0,stop=0):
    for i in range(start,stop):
        print(f"{i:4}.  --{df.loc[i].text} -- {df.loc[i].clean_text}")

In [ ]:
print_df(df,1,10)

In [126]:
def create_all_sentence(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    return(all_sent)


In [134]:
all_sent=create_all_sentence(df)
df["sentence_code"]=all_sent

In [135]:

all_sent=create_all_sentence(df_pos_neutral)

In [136]:
df_pos_neutral["sentence_code"]=all_sent

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [137]:
df_pos_neutral.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
1,568830255282040832,neutral,1.0,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana,i will do you now,"[4273, 9420, 2763, 9586, 5846]"


In [138]:
table=list(df["sentence_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df_pos_neutral["airline_sentiment"])


In [139]:
def ylabel_convert(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0]
        if sent=="neutral":
            o=[0.5,0.5]
        if sent=="positive":
            o=[0,1]
        ytrain.append(o)
    return(ytrain)

def ylabel_convert3(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0,0]
        if sent=="neutral":
            o=[0,1,0]
        if sent=="positive":
            o=[0,0,1]
        ytrain.append(o)
    return(ytrain)

def ylabel_convert2(sentiment):
    ytrain=[]
    for sent in sentiment:
      
        if sent=="neutral":
            o=[1,0]
        if sent=="positive":
            o=[0,1]
        ytrain.append(o)
    return(ytrain)


        

In [140]:
ytrain=ylabel_convert2(sentiment)

In [150]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in df_pos_neutral["sentence_code"]:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [151]:
x_train=xtrain[0:-20]
y_train=ytrain[0:-20]
x_test=xtrain[-20:]
y_test=ytrain[-20:]

In [152]:
len(x_train)

4463

In [153]:


lstm_size=lens
max_input_length=lens
embedding_size=190 #(100: 73%)
n_words=len(table)
n_out=len(y_train[0])

In [154]:
lens
n_out

2

In [155]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed, Flatten, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy, mean_squared_error
from tensorflow.keras import backend as K

K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output= LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output= LSTM(lstm_size,return_sequences=True)(lstm_output)


lstm_output=Flatten()(lstm_output)
#Dense_out= Dense(33, activation="softmax")(lstm_output)

predictions= Dense(n_out, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions,)



In [156]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 59)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 59, 190)           2280190   
_________________________________________________________________
lstm_2 (LSTM)                (None, 59, 59)            59000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 59, 59)            28084     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3481)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6964      
Total params: 2,374,238
Trainable params: 2,374,238
Non-trainable params: 0
_________________________________________________

In [157]:
import numpy as np

In [ ]:
#train_x = np.asarray(xtrain)
#train_y = np.asarray(ytrain)

In [158]:
# Loss 

loss = mean_squared_error #categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [159]:
# Illesszük az adatra a modellt.
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=6, batch_size=50,)


Epoch 1/6
90/90 [==============================] - 23s 177ms/step - loss: 0.1728 - val_loss: 0.0856
Epoch 2/6
90/90 [==============================] - 14s 150ms/step - loss: 0.0864 - val_loss: 0.0415
Epoch 3/6
90/90 [==============================] - 14s 150ms/step - loss: 0.0599 - val_loss: 0.0259
Epoch 4/6
90/90 [==============================] - 14s 151ms/step - loss: 0.0423 - val_loss: 0.0141
Epoch 5/6
90/90 [==============================] - 13s 150ms/step - loss: 0.0341 - val_loss: 0.0149
Epoch 6/6
90/90 [==============================] - 13s 150ms/step - loss: 0.0292 - val_loss: 0.0112


In [ ]:
pred=model.predict(x_train)

In [ ]:
for i in range(len(pred)):
    print(f"{i}, {pred[i]}, {y_train[i]}")



### Model usage

In [ ]:
test_df=pd.read_csv("TEST.csv")

In [ ]:
print(len(test_df))

In [ ]:
test_df.head(130)

In [ ]:
test_df["clean_text"]=sentences_list[:2640]

In [ ]:
def code_sentences(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    df["sentence_code"]=all_sent
    return (df)

In [ ]:
test_df=code_sentences(test_df)

In [ ]:
def push_sentences(df,lens=33):
    table=list(df["sentence_code"]) # typo
    
    print(lens)
    x0=[0 for _ in range(lens) ]
    x=[]
    for sent in table:
        o1=list(x0[0:lens-len(sent)]+list(sent))
        x.append(o1)
    return(x)

In [ ]:
test_df.head(1)

In [ ]:

xx=push_sentences(test_df,lens=59)

In [ ]:
test_out=model.predict(xx)

In [ ]:
test_out

In [ ]:
def decode3(model_out):
    y=[]
    for mo in model_out:
        if mo[0]>mo[1] and mo[0]>mo[2]:
            o="negative"    
        if mo[1]>mo[0] and mo[1]>mo[2]:
            o="neutral"    
        if mo[2]>mo[0] and mo[2]>mo[1]:
            o="positive"    
        y.append(o)
    return(y)


  

In [ ]:
def decode(model_out):
    y=[]
    for mo in model_out:
        dif=abs(mo[0]-mo[1])
        if dif < 0.2 :
            o="neutral"
        else:
            if mo[0]>mo[1] :
                o="negative"    
            else:
                o="positive"    
        y.append(o)
    return(y)



In [ ]:
prediction=decode3(test_out)

In [ ]:
def listing(status):
    counti=0
    for i in range(len(prediction)):
    
        predi=prediction[i]
        reason=test_df.loc[i].negativereason
        airline_sentiment=test_df.loc[i].airline_sentiment_gold
        if predi==status:
            counti+=1
            print(f"{i:4}. {counti:3}. =  {prediction[i]} -- {reason:15}--{airline_sentiment} -- {test_df.loc[i].text}")
            cond=[reason=="Flight Booking Problems", 
                    reason=="Bad Flight", 
                    reason=="Late Flight", 
                    reason=="Lost Luggage", 
                    reason=="Customer Service Issue",
                    reason=="Cancelled Flight",
                    reason=="Flight Attendant Complaints",
                    reason=="longlines",
                    reason=="Damaged Luggage"
                    ]
            if any(cond):
                prediction[i]="negative"


In [ ]:
df.head(1)

In [ ]:
def retweet_converter():
    counti=0
    for i in range(len(prediction)):
    
        predi=prediction[i]
        reason=test_df.loc[i].negativereason
        retweet=test_df.loc[i].retweet_count
        if retweet>0:
            prediction[i]="negative"

In [ ]:
listing("neutral")
#retweet_converter()

In [ ]:
test_df["predicted"]=prediction

In [ ]:
prediction_df=pd.DataFrame(prediction)

In [ ]:
prediction_df.columns=["Sentiment"]

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.index.name="Index"

In [ ]:
from datetime  import datetime

In [ ]:
a=datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
print(a)

In [ ]:
prediction_df.to_csv("submission_"+a+".csv")

In [ ]:
predcsv=test_df.loc[:,["text","predicted"]]

predcsv.to_csv("mini.csv")

In [ ]:
print_df(test_df,95,145)

In [ ]:
def de_ex(txt):
    txt=txt.lower()
    exwords={"it's":"it is",
    "aren't":"are not",
    "i've":"i have",
    "can't":"can not",
    "didn't":"did not",
    "i'm":"i am",
    "that's":"that is",
    "let's":" let is",
    "she's":"she is",
    "we're":"we are",
    "they've":"they have",
    "fleet's":"fleet is ",
    "there's":"there is"
    "i'll": "i will"
    }
    txt_list=txt.split()
    for w in txt_list:
        if w in exwords:
            txt=txt.replace(w,exwords[w])
    return (txt)


In [ ]:
de_ex("hello i'm bond, james bond")

In [ ]:
def print_char_df(df,character,start=0,stop=0):
    for i in range(start,stop):
        if character in df.loc[i].text:
            print(f"{i:4}.  --{df.loc[i].text} -- {df.loc[i].clean_text}")

In [ ]:
print_char_df(test_df,"'",0,100)